In [1]:
def my_decorator(some_function):

    def wrapper():

        num = 10

        if num == 10:
            print("Yes!")
        else:
            print("No!")

        some_function()

        print("Something is happening after some_function() is called.")

    return wrapper


In [22]:
from energy_py import make_paths, make_logger, EternityVisualizer
from energy_py.envs import BatteryEnv
from energy_py.agents import RandomAgent

In [25]:
def experiment():
    def outer_wrapper(my_expt_func):

        def wrapper(**kwargs):
            LOG_STATUS = False
            EPISODE_LENGTH = 10
            EPISODES = 20
            EPISODE_RANDOM = False
            OUTPUT_RESULTS = 5
            DISCOUNT=0.9

            data_path = kwargs['data_path']
            base_path = kwargs['base_path']
            env = kwargs['env']
            agent = kwargs['agent']

            print('before agent expt')
            paths = make_paths(base_path)
            BRAIN_PATH = paths['brain']
            RESULTS_PATH = paths['results']
            ARGS_PATH = paths['args']
            LOG_PATH = paths['logs']

            logger = make_logger(LOG_PATH, LOG_STATUS)

            env = env(data_path, 
                      episode_length=EPISODE_LENGTH,
                      episode_random=EPISODE_RANDOM)
            
            agent_outputs, env_outputs = my_expt_func(env, agent, 
                                                      EPISODES, DISCOUNT, OUTPUT_RESULTS,
                                                      RESULTS_PATH)

            return agent_outputs, env_outputs
        
        return wrapper
    return outer_wrapper

@experiment()
def no_learn_expt(env, agent, EPISODES, DISCOUNT, OUTPUT_RESULTS, RESULTS_PATH):
    
    agent = agent(env, DISCOUNT)
    
    for episode in range(1, EPISODES):

        #  initialize before starting episode
        done, step = False, 0
        observation = env.reset(episode)

        #  while loop runs through a single episode
        while done is False:
            #  we cheat a little bit here by grabbing the observation as a
            #  pandas series
            #  just want to pull out the timestamp
            time = env.observation_ts.index[env.steps]
            #  select an action
            action = agent.act(observation=observation, timestamp=time)
            #  take one step through the environment
            next_observation, reward, done, info = env.step(action)
            #  store the experience
            agent.memory.add_experience(observation, action, reward,
                                        next_observation, done, step, episode)
            step += 1
            observation = next_observation

        if episode % OUTPUT_RESULTS == 0:
            #  collect data from the agent & environment
            hist = EternityVisualizer(agent,
                                      env,
                                      results_path=RESULTS_PATH)

            agent_outputs, env_outputs = hist.output_results(save_data=True)
    return agent_outputs, env_outputs

            

In [26]:
env = no_learn_expt(data_path='', base_path='base', env=BatteryEnv, agent=RandomAgent)

2017-12-17 22:48:38,824 [DEBUG]energy_py.envs.env_core: Reset environment
2017-12-17 22:48:38,827 [INFO]energy_py.envs.env_ts: Ep none starting at 2017-01-01 00:00:00
2017-12-17 22:48:38,828 [DEBUG]energy_py.envs.battery.battery_env: initial state is [[ 40.   0.]]
2017-12-17 22:48:38,829 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-17 22:48:38,830 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-17 22:48:38,831 [DEBUG]energy_py.envs.env_core: Reset environment
2017-12-17 22:48:38,834 [INFO]energy_py.envs.env_ts: Ep 1 starting at 2017-01-01 00:00:00
2017-12-17 22:48:38,835 [DEBUG]energy_py.envs.battery.battery_env: initial state is [[ 40.   0.]]
2017-12-17 22:48:38,836 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-17 22:48:38,837 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-17 22

2017-12-17 22:48:38,914 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.854 MW
2017-12-17 22:48:38,915 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.007 MWh
2017-12-17 22:48:38,916 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.768 MW
2017-12-17 22:48:38,917 [DEBUG]energy_py.envs.battery.battery_env: reward is -2.845 $/5min
2017-12-17 22:48:38,918 [DEBUG]energy_py.agents.memory: adding exp episode 1 step 7
2017-12-17 22:48:38,920 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:38,920 [DEBUG]energy_py.envs.env_core: Episode 1 - Step 8
2017-12-17 22:48:38,921 [DEBUG]energy_py.envs.battery.battery_env: step is 8.000
2017-12-17 22:48:38,922 [DEBUG]energy_py.envs.battery.battery_env: action was [ 1.62186011  0.53537619]
2017-12-17 22:48:38,923 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.064 MWh
2017-12-17 22:48:38,923 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.146 MWh
2017-12-17 22:48:38,924 [DEBUG]energy_py.env

2017-12-17 22:48:39,015 [DEBUG]energy_py.envs.battery.battery_env: gross rate is -0.382 MW
2017-12-17 22:48:39,016 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-17 22:48:39,017 [DEBUG]energy_py.envs.battery.battery_env: net rate is -0.382 MW
2017-12-17 22:48:39,018 [DEBUG]energy_py.envs.battery.battery_env: reward is 1.274 $/5min


before agent expt


2017-12-17 22:48:39,019 [DEBUG]energy_py.agents.memory: adding exp episode 2 step 6
2017-12-17 22:48:39,021 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:39,022 [DEBUG]energy_py.envs.env_core: Episode 2 - Step 7
2017-12-17 22:48:39,023 [DEBUG]energy_py.envs.battery.battery_env: step is 7.000
2017-12-17 22:48:39,025 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.49689624  1.90933466]
2017-12-17 22:48:39,026 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.021 MWh
2017-12-17 22:48:39,029 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.000 MWh
2017-12-17 22:48:39,030 [DEBUG]energy_py.envs.battery.battery_env: gross rate is -0.253 MW
2017-12-17 22:48:39,031 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-17 22:48:39,038 [DEBUG]energy_py.envs.battery.battery_env: net rate is -0.253 MW
2017-12-17 22:48:39,039 [DEBUG]energy_py.envs.battery.battery_env: reward is 0.844 $/5min
2017-12-17 22:48:39,040 [DEBUG]energy_py.ag

...

2017-12-17 22:48:44,421 [DEBUG]energy_py.envs.env_core: Episode 7 - Step 7
2017-12-17 22:48:44,422 [DEBUG]energy_py.envs.battery.battery_env: step is 7.000
2017-12-17 22:48:44,423 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.03299923  0.54255802]
2017-12-17 22:48:44,425 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.000 MWh
2017-12-17 22:48:44,425 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.000 MWh
2017-12-17 22:48:44,426 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.000 MW
2017-12-17 22:48:44,427 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-17 22:48:44,428 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.000 MW
2017-12-17 22:48:44,429 [DEBUG]energy_py.envs.battery.battery_env: reward is -0.000 $/5min
2017-12-17 22:48:44,430 [DEBUG]energy_py.agents.memory: adding exp episode 7 step 7
2017-12-17 22:48:44,432 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:44,433 [DEBUG]energy_py.env

2017-12-17 22:48:44,529 [DEBUG]energy_py.envs.env_core: Episode 8 - Step 6
2017-12-17 22:48:44,529 [DEBUG]energy_py.envs.battery.battery_env: step is 6.000
2017-12-17 22:48:44,530 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.34828828  0.05148818]
2017-12-17 22:48:44,531 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.095 MWh
2017-12-17 22:48:44,531 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.117 MWh
2017-12-17 22:48:44,532 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.297 MW
2017-12-17 22:48:44,533 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.002 MWh
2017-12-17 22:48:44,533 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.267 MW
2017-12-17 22:48:44,534 [DEBUG]energy_py.envs.battery.battery_env: reward is -0.989 $/5min
2017-12-17 22:48:44,535 [DEBUG]energy_py.agents.memory: adding exp episode 8 step 6
2017-12-17 22:48:44,536 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:44,537 [DEBUG]energy_py.env

2017-12-17 22:48:44,629 [DEBUG]energy_py.envs.env_core: Episode 9 - Step 5
2017-12-17 22:48:44,630 [DEBUG]energy_py.envs.battery.battery_env: step is 5.000
2017-12-17 22:48:44,631 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.2802922   0.02223366]
2017-12-17 22:48:44,632 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.115 MWh
2017-12-17 22:48:44,632 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.135 MWh
2017-12-17 22:48:44,633 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.258 MW
2017-12-17 22:48:44,634 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.002 MWh
2017-12-17 22:48:44,634 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.232 MW
2017-12-17 22:48:44,635 [DEBUG]energy_py.envs.battery.battery_env: reward is -0.860 $/5min
2017-12-17 22:48:44,636 [DEBUG]energy_py.agents.memory: adding exp episode 9 step 5
2017-12-17 22:48:44,637 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:44,638 [DEBUG]energy_py.env

2017-12-17 22:48:44,705 [DEBUG]energy_py.envs.env_core: Episode 10 - Step 4
2017-12-17 22:48:44,706 [DEBUG]energy_py.envs.battery.battery_env: step is 4.000
2017-12-17 22:48:44,706 [DEBUG]energy_py.envs.battery.battery_env: action was [ 1.0745114   0.19796646]
2017-12-17 22:48:44,707 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.130 MWh
2017-12-17 22:48:44,708 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.195 MWh
2017-12-17 22:48:44,709 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.877 MW
2017-12-17 22:48:44,709 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.007 MWh
2017-12-17 22:48:44,710 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.789 MW
2017-12-17 22:48:44,710 [DEBUG]energy_py.envs.battery.battery_env: reward is -2.922 $/5min
2017-12-17 22:48:44,711 [DEBUG]energy_py.agents.memory: adding exp episode 10 step 4
2017-12-17 22:48:44,712 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-17 22:48:44,713 [DEBUG]energy_py.e

SystemError: <built-in method write of _io.BufferedWriter object at 0x7f761d264888> returned a result with an error set

{'CLICOLOR': '1',
 'CLUTTER_IM_MODULE': 'xim',
 'COMPIZ_CONFIG_PROFILE': 'ubuntu',
 'CONDA_DEFAULT_ENV': 'energy_py',
 'CONDA_PATH_BACKUP': '/home/adam/anaconda3/bin:/home/adam/bin:/home/adam/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin',
 'CONDA_PREFIX': '/home/adam/anaconda3/envs/energy_py',
 'CONDA_PS1_BACKUP': '\\[\\e]0;\\u@\\h: \\w\\a\\]${debian_chroot:+($debian_chroot)}\\[\\033\\]\\u@\\h\\[\\033\\]:\\[\\033\\]\\w\\[\\033\\]\\$ ',
 'CPLUS_INCLUDE_PATH': ':/home/adam/git/Arcade-Learning-Environment/src',
 'CUDA_HOME': '/usr/local/cuda',
 'DBUS_DEBUG_OUTPUT': '1',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:abstract=/tmp/dbus-40Uf3TtTRw,guid=8f7cd9368b56c2f328a671a75a36a140',
 'DBUS_STARTER_ADDRESS': 'unix:abstract=/tmp/dbus-40Uf3TtTRw,guid=8f7cd9368b56c2f328a671a75a36a140',
 'DBUS_STARTER_BUS_TYPE': 'session',
 'DEFAULTS_PATH': '/usr/share/gconf/ubuntu.default.path',
 'DESKTOP_SESSION': 'ubuntu',
 'DISPLAY': ':0',
 'GDMSESSION